In [1]:
# Create an ImageJ gateway
import imagej
import functools

In [2]:
# pyimagej initialisation: https://github.com/imagej/pyimagej

imagej_core_dep = 'net.imagej:imagej:2.1.0' 
scijava_command_dep = 'ch.epfl.biop:pyimagej-scijava-command:0.1.2'
legacy_dep = 'net.imagej:imagej-legacy:0.37.4'

ij = imagej.init([imagej_core_dep, legacy_dep, scijava_command_dep], headless=False)

In [3]:
# Import scijava classes using jimport
from scyjava import jimport
Command = jimport('org.scijava.command.Command')
JPlugin = jimport('org.scijava.plugin.Plugin')
Parameter = jimport('org.scijava.plugin.Parameter')
ImagePlus = jimport('ij.ImagePlus')

In [4]:
# jpype documentation: https://jpype.readthedocs.io/en/latest/

#jpype imports
import jpype
from jpype import JImplements, JOverride

# Pull in types
from jpype.types import *

In [5]:
# Different class which can be used as input or output parameter
# points to java object
print(JInt)
print(JInt.class_)
print(JBoolean)
print(JBoolean.class_)
print(ImagePlus)
print(ImagePlus.class_)
print(JString)
print(JString.class_)

# Not sure I understand
print(type(JInt))

# The class of a class is Class, ok
print(type(JInt.class_))

<java class 'JInt'>
int
<java class 'JBoolean'>
boolean
<java class 'ij.ImagePlus'>
class ij.ImagePlus
<java class 'JString'>
class java.lang.String
<class '_jpype._JClass'>
<java class 'java.lang.Class'>


In [6]:
#org.scijava.command.PyCommandBuilder
PyCommandBuilder = jimport('org.scijava.command.PyCommandBuilder')

# Decorator that registers a python CLASS containing a method named "run" as a scijava Command
#
# This uses PyCommandBuilder which is in the java repo ch.epfl.biop:pyimagej-scijava-command:0.1.0-SNAPSHOT
# PyCommandBuilder allows to build a Command fully programmatically without using any
# java annotation as java annotations are needed for 'easy' Scijavy Commands definition
# but these are not completely supported in JPype: 
# cf https://github.com/jpype-project/jpype/issues/940
# 
# See cell below for example usage
#
# Note: this way of defining a command is probably not ideal if this has to be used from the python side also
#
# Because it's a preliminary work, this decorator prints a lot of stuff in the process
#
# TODO: functools ?? NOt sure it's necessary

def ScijavaCommand(**kwargs):    
    
    print("- Registering scijava command "+kwargs['name'])
    
    def registerCommand(func): 
        # This class will be registered as a SciJava Command
        builder = PyCommandBuilder() # Java PyCommandBuilder
        
        # The name of the command - to avoid name conflicts, consider a 'virtual' class name with its package
        builder = builder.name(kwargs['name'])
        
        # Register all inputs
        print('- Inputs')
        for name, javaClass in kwargs['inputs'].items():
            print('\t', name,' : ', javaClass)
            builder = builder.input(name, javaClass)
            setattr(func, name, None) # declares empty input field 
        print('Inputs registered')
        
        # Register all outputs
        print('- Outputs')
        for name, javaClass in kwargs['outputs'].items():
            print('\t', name,' : ', javaClass)
            builder = builder.output(name, javaClass)
            setattr(func, name, None) # declares empty output field
        print('Outputs registered')
                
        # Wraps the run function - takes kwargs as input, returns outputs
        def wrapped_run(inner_kwargs):
            inner_object = func()
            print('Settings inputs...')
            print(inner_kwargs)
            for name, javaClass in kwargs['inputs'].items():
                #print(name)
                #print(str(inner_kwargs[name]))
                setattr(inner_object, name, inner_kwargs[name]) # sets inputs 
            print('Inputs set.')
            print('Running scijava command: '+kwargs['name'])
            inner_object.run() #I'm not sure this actually works
            print(kwargs['name']+' command execution done.')
            print('Fetching outputs...')
            outputs = {}
            for name, javaClass in kwargs['outputs'].items():
                outputs[name] = getattr(inner_object, name) # gets outputs
            print('Outputs set.')
            return JObject(outputs, JClass('java.util.Map')) # Returns output as a java HashMap
        
        # Sets the function in PyCommandBuilder:
        # Function<Map<String, Object>, Map<String, Object>> command
        builder = builder.function(wrapped_run)
        
        # Effectively registers this command to the ij context
        builder.create(kwargs['context']) 
        return func
    
    return registerCommand

In [7]:
# Example of registering a Scijava Command via the @ScijavaCommand decorator

@ScijavaCommand(context = ij.context(), # ij context needed
                name = 'pyCommand.HelloCommand', # name of this command, mind potential naming conflicts!
                inputs = {'name': JString, 'familiar': JBoolean}, # input name, input Java class, as dictionary
                outputs = {'greetings': JString}) # output name, output Java class, as dictionary
class MyPyCommand:        
    def run(self):
        if (self.familiar):
            self.greetings = 'Hi ' + str(self.name) + '!'
        else:
            self.greetings = 'Hello my dear ' + str(self.name) + '.'


- Registering scijava command pyCommand.HelloCommand
- Inputs
	 name  :  <java class 'JString'>
	 familiar  :  <java class 'JBoolean'>
Inputs registered
- Outputs
	 greetings  :  <java class 'JString'>
Outputs registered


In [8]:
# Now we can start the ui of ImageJ
# look for and execute the command 'pyCommand.HelloCommand' in the search bar
ij.ui().showUI()

# pyCommand.HelloCommand can be recorded in ImageJ and executed using a command service, 
# either in groovy (or jython) scripting in ImageJ, or in this notebook as shown below

Settings inputs...
{name=Bob, familiar=true}
Inputs set.
Running scijava command: pyCommand.HelloCommand
pyCommand.HelloCommand command execution done.
Fetching outputs...
Outputs set.


In [9]:
ij.command().run('pyCommand.HelloCommand', True) # no args -> all inputs required, check the UI!

Settings inputs...
{name=Bob, familiar=true}
Inputs set.
Running scijava command: pyCommand.HelloCommand
pyCommand.HelloCommand command execution done.
Fetching outputs...
Outputs set.


In [10]:
ij.command().run('pyCommand.HelloCommand', True, 'name', 'Bob') # familiar still required

Settings inputs...
{name=Bob, familiar=false}
Inputs set.
Running scijava command: pyCommand.HelloCommand
pyCommand.HelloCommand command execution done.
Fetching outputs...
Outputs set.


In [10]:
module = ij.command().run('pyCommand.HelloCommand', True, 'name', 'Bob', 'familiar', True) # all args given


# nodule returns None: Weird : how can I retrieve the output ? Or maybe it's eaten by the GUI ?

Settings inputs...
{name=Bob, familiar=true}
Inputs set.
Running scijava command: pyCommand.HelloCommand
pyCommand.HelloCommand command execution done.
Fetching outputs...
Outputs set.


In [ ]:
# THIS CELL WON'T RUN
# Idea, another way of registering of Command could be like this:

@ScijavaFunctionDecorator
def myCommand(name: JString, familiar:JBoolean)
    greetings = "Hello" + name 
    return {'greetings':greetings, another param as a dictionary}

# But I'm not sure the decorator can be aware of the 'output' signature, thus the decorator
# will have a hard time, (maybe impossible?) to figure out the outputs and their class